<a href="https://colab.research.google.com/github/joaquiimfreitas/order-caller/blob/main/Machine_Learning_Valida%C3%A7%C3%A3o_de_Modelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**CROSS VALIDATING**



**Abrindo e tratando o arquivo**

In [1]:
import pandas as pd

uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"
dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


In [2]:
dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)

In [3]:
x = dados[["preco", "idade_do_modelo", "km_por_ano"]]
y = dados["vendido"]

**Realizando o Split com o train_test_split e realizando o fit com o LinearSVC**

In [4]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

x = dados[["preco", "idade_do_modelo","km_por_ano"]]
y = dados["vendido"]

SEED = 158020
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

Treinaremos com 7500 elementos e testaremos com 2500 elementos


**Validando com o Dummy**

In [5]:
from sklearn.dummy import DummyClassifier

dummy_stratified = DummyClassifier(strategy='stratified')
dummy_stratified.fit(treino_x, treino_y)
acuracia = dummy_stratified.score(teste_x, teste_y) * 100

print("A acurácia do dummy stratified foi de %.2f%%" % acuracia)

A acurácia do dummy stratified foi de 50.96%


**Treinando com o DecisionTreeClassifier**

In [6]:
from sklearn.tree import DecisionTreeClassifier

SEED = 158020
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print ("A acurácia foi %.2f%%" % acuracia)

A acurácia foi 71.92%


**Realizando o CrossValidate**

In [7]:
from sklearn.model_selection import cross_validate

SEED = 158020
np.random.seed(SEED)
tam_cv = 5

modelo = DecisionTreeClassifier(max_depth=2)
cvdt = cross_validate(modelo, x, y, cv=tam_cv, return_train_score=False)
media = cvdt['test_score'].mean()
std = cvdt['test_score'].std()

print('A acurácia com um cross validation com cv = %d é %.2f%%, +- %.2f%%' %(tam_cv, (media*100),(std*100)))

A acurácia com um cross validation com cv = 5 é 75.78%, +- 0.29%


#**KFold com aleatorização**

**Aleatoriedade no cross validate**

In [8]:
def set_msg(cvdt, n_splits):
  media = cvdt['test_score'].mean()
  std = cvdt['test_score'].std()

  print('A acurácia média com um cross validation de cv = %d é: %.2f%%, \nO intervalo é de %.2f%% até %.2f%%' %(int(n_splits), (media*100),(media*100 - std*100), (media*100 + std*100)))

In [9]:
from sklearn.model_selection import KFold

SEED = 301
np.random.seed(SEED)

n_splits = 10
cv = KFold(n_splits=n_splits)

modelo = DecisionTreeClassifier(max_depth=2)
cvdt = cross_validate(modelo, x, y, cv = cv)

set_msg(cvdt, n_splits)

A acurácia média com um cross validation de cv = 10 é: 75.78%, 
O intervalo é de 75.08% até 76.48%


In [10]:
from sklearn.model_selection import KFold

SEED = 301
np.random.seed(SEED)

n_splits = 10
cv = KFold(n_splits=n_splits, shuffle=True)

modelo = DecisionTreeClassifier(max_depth=2)
cvdt = cross_validate(modelo, x, y, cv = cv)

set_msg(cvdt, n_splits)

A acurácia média com um cross validation de cv = 10 é: 75.76%, 
O intervalo é de 74.51% até 77.01%


In [11]:
SEED = 301
shuffle=True
n_splits=10

def treinador(SEED, n_splits, shuffle, x, y):
  np.random.seed(SEED)
  cv = KFold(n_splits=n_splits, shuffle=shuffle)

  modelo = DecisionTreeClassifier(max_depth=2)
  cvdt = cross_validate(modelo, x, y, cv = cv)

  set_msg(cvdt, n_splits)

treinador(SEED, n_splits, shuffle, x, y)

A acurácia média com um cross validation de cv = 10 é: 75.76%, 
O intervalo é de 74.51% até 77.01%


**Simulando situação de azar/desbalanceamento dos dados**

In [12]:
dados_azar = dados.sort_values('vendido', ascending=True)
x_azar = dados_azar[['preco','idade_do_modelo','km_por_ano']]
y_azar = dados_azar['vendido']

x_azar, y_azar

(         preco  idade_do_modelo   km_por_ano
 4999  74023.29               12  24812.80412
 5322  84843.49               13  23095.63834
 5319  83100.27               19  36240.72746
 5316  87932.13               16  32249.56426
 5315  77937.01               15  28414.50704
 ...        ...              ...          ...
 5491  71910.43                9  25778.40812
 1873  30456.53                6  15468.97608
 1874  69342.41               11  16909.33538
 5499  70520.39               16  19622.68262
 0     30941.02               18  35085.22134
 
 [10000 rows x 3 columns], 4999    0
 5322    0
 5319    0
 5316    0
 5315    0
        ..
 5491    1
 1873    1
 1874    1
 5499    1
 0       1
 Name: vendido, Length: 10000, dtype: int64)

In [13]:
shuffle = True
treinador(SEED, n_splits, shuffle, x_azar, y_azar)

A acurácia média com um cross validation de cv = 10 é: 75.78%, 
O intervalo é de 74.04% até 77.52%


#**Dados Agrupados**

**Gerando dados aleatórios**

In [14]:
np.random.seed(SEED)
dados['modelo'] = dados['idade_do_modelo'] + np.random.randint(-2, 3, size=len(dados))

In [15]:
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo
0,30941.02,1,18,35085.22134,16
1,40557.96,1,20,12622.05362,22
2,89627.50,0,12,11440.79806,12
3,95276.14,0,3,43167.32682,4
4,117384.68,1,4,12770.11290,3


In [16]:
abs(dados.modelo.min())

1

In [17]:
dados['modelo'] = dados['modelo'] + abs(dados.modelo.min()) + 1
dados.modelo.unique()

array([18, 24, 14,  6,  5, 13, 20, 19, 15,  2, 17, 12, 11, 16,  3,  7, 21,
       23, 10,  9, 22,  8,  4,  1])

In [18]:
dados.modelo.value_counts()

20    901
19    798
18    771
21    723
17    709
16    668
14    621
22    575
15    573
13    557
12    511
11    401
10    371
23    370
9     336
8     278
7     206
24    199
6     181
5     108
4      76
3      44
2      17
1       6
Name: modelo, dtype: int64

**Group KFold**

In [24]:
from sklearn.model_selection import GroupKFold

SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits=10)

modelo = DecisionTreeClassifier(max_depth=2)
cvdt = cross_validate(modelo, x, y, cv=cv, groups = dados.modelo)

set_msg(cvdt, 5)

A acurácia média com um cross validation de cv = 5 é: 75.80%, 
O intervalo é de 73.90% até 77.70%


#**Cross validation com Standard Scaler**

In [26]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(treino_x)
treino_x_s = scaler.transform(treino_x)
teste_x_s = scaler.transform(teste_x)

In [28]:
from sklearn.svm import SVC

np.random.seed(SEED)

modelo = SVC()
modelo.fit(treino_x_s, treino_y)
previsoes = modelo.predict(teste_x_s)

acuracia = accuracy_score(teste_y, previsoes) * 100
print ("A acurácia foi %.2f%%" % acuracia)

A acurácia foi 74.40%


In [29]:
cv = GroupKFold(n_splits=10)

modelo = SVC()
cvdt = cross_validate(modelo, x, y, cv=cv, groups = dados.modelo)

set_msg(cvdt, 5)

A acurácia média com um cross validation de cv = 5 é: 77.23%, 
O intervalo é de 75.49% até 78.96%


**Pipeline** </br>
Um processo

In [34]:
from sklearn.pipeline import Pipeline

scaler = StandardScaler()
modelo = SVC()

pipeline = Pipeline([('transformacao',scaler), ('estimador',modelo)])

In [35]:
cv = GroupKFold(n_splits=10)

modelo = SVC()
cvdt = cross_validate(pipeline, x_azar, y_azar, cv=cv, groups = dados.modelo)

set_msg(cvdt, 5)

A acurácia média com um cross validation de cv = 5 é: 76.68%, 
O intervalo é de 75.48% até 77.88%
